In [1]:
import argparse
import qlib
import ruamel.yaml as yaml
from qlib.utils import init_instance_by_config
import pandas as pd
import torch
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)

In [2]:
parser = argparse.ArgumentParser(allow_abbrev=False)
parser.add_argument("--seed", type=int, default=1000, help="random seed")
parser.add_argument("--config_file", type=str, default="backtest.yaml", help="config file")
parser.add_argument("-f")
args = parser.parse_args()


seed = args.seed
config_file = args.config_file

# set random seed
with open(config_file) as f:
    config = yaml.safe_load(f)

# seed_suffix = "/seed1000" if "init" in config_file else f"/seed{seed}"
seed_suffix = ""
config["task"]["model"]["kwargs"].update(
    {"seed": seed, "logdir": config["task"]["model"]["kwargs"]["logdir"] + seed_suffix}
)

# initialize workflow
qlib.init(
    provider_uri=config["qlib_init"]["provider_uri"],
    region=config["qlib_init"]["region"],
)

[2410:MainThread](2021-10-27 14:43:43,812) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[2410:MainThread](2021-10-27 14:43:43,817) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[2410:MainThread](2021-10-27 14:43:44,023) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[2410:MainThread](2021-10-27 14:43:44,024) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data')}


In [3]:
latest_dataset = init_instance_by_config(config["task"]["dataset"])

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[2410:MainThread](2021-10-27 14:47:08,709) INFO - qlib.timer - [log.py:113] - Time cost: 202.509s | Loading data Done
[2410:MainThread](2021-10-27 14:47:11,320) INFO - qlib.timer - [log.py:113] - Time cost: 2.248s | FilterCol Done
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.8/dist

In [4]:
model = init_instance_by_config(config["task"]["model"])

Please install necessary libs for CatBoostModel.
Please install necessary libs for XGBModel, such as xgboost.


[2410:MainThread](2021-10-27 14:47:28,598) WARNING - qlib.TRA - [pytorch_tra.py:100] - `eval_train` will be ignored when using TRA.router
[2410:MainThread](2021-10-27 14:47:28,601) WARNING - qlib.TRA - [pytorch_tra.py:132] - logdir output/us_158_nassp_epoch100 is not empty
[2410:MainThread](2021-10-27 14:47:28,603) INFO - qlib.TRA - [pytorch_tra.py:141] - init TRAModel...
[2410:MainThread](2021-10-27 14:47:28,611) INFO - qlib.TRA - [pytorch_tra.py:171] - # model params: 59520
[2410:MainThread](2021-10-27 14:47:28,613) INFO - qlib.TRA - [pytorch_tra.py:172] - # tra params: 5606


RNN(
  (rnn): LSTM(20, 64, num_layers=2, batch_first=True)
  (W): Linear(in_features=64, out_features=64, bias=True)
  (u): Linear(in_features=64, out_features=1, bias=False)
)
TRA(
  (predictors): Linear(in_features=128, out_features=3, bias=True)
  (router): LSTM(3, 32, batch_first=True)
  (fc): Linear(in_features=160, out_features=3, bias=True)
)


In [5]:
state_dict = torch.load("/home/qlib_test/examples/benchmarks/TRA/output/nasdaq/model.bin", map_location="cpu")["model"]

In [6]:
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [7]:
model.fitted = True

In [8]:
pred = model.predict(latest_dataset)

100%|████████████████████████████████████████████████████| 1405/1405 [03:18<00:00,  7.07it/s]
[2410:MainThread](2021-10-27 14:50:48,247) INFO - qlib.TRA - [pytorch_tra.py:516] - test metrics: {'MSE': 0.16775067091895768, 'MAE': 0.33403291976234856, 'IC': -0.006520966394332231, 'ICIR': -0.0725018076913665}


In [9]:
import qlib
from qlib.data import D
qlib.init(provider_uri="/home/us_data", region="us")

[2410:MainThread](2021-10-27 14:50:48,266) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[2410:MainThread](2021-10-27 14:50:48,268) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[2410:MainThread](2021-10-27 14:50:48,271) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[2410:MainThread](2021-10-27 14:50:48,273) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data')}


In [10]:
df = D.features(D.instruments("nassp"), ["$open"], freq="day")

In [11]:
past_data = df.reset_index('instrument')

In [42]:
today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].values.round(2)

AttributeError: 'numpy.float32' object has no attribute 'values'

In [85]:
temp = past_data[past_data['instrument'] == 'GAS']
ak = temp.loc['2015-01-02']['$open'].round(6)

In [86]:
np.isnan(ak)

True

In [13]:
reset_df = pred.reset_index('instrument')

In [89]:
from IPython.display import clear_output
import numpy as np
import os
import time

indexs = reset_df.index.drop_duplicates()
cnt = 0 

money = 100
current_buying = {}
top_k = 5
max_buying = 10
num_buy = 0
# num_sell = 0
my_stack = None

profit = 0.0

data_dir = '/home/us_data_source/'
for idx in indexs:
    cnt+=1
    
    # if cnt < 800: continue
    # today_inform = pd.read_csv(results_dir + str(idx) +'.csv')
    # print(today_inform)
    if type(reset_df.loc[idx]['score']) is np.float32:
        stack = reset_df.loc[idx]
    else:
        stack = reset_df.loc[idx].sort_values(by='score' , ascending=False)
        
    # st = time.time()
    current_buying_list = list(current_buying.keys())
    for cur_stock in current_buying_list:
        cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
        today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
        # if today_stock_price is np.nan:
        # print(np.isnan(today_stock_price))
        # else:
        current_buying[cur_stock][2] = today_stock_price
    
    # print("2", time.time() - st)
    
    my_stack = stack[stack['instrument'].isin(current_buying_list)]
    my_stack = my_stack.sort_values(by='score' , ascending=False)
    my_stack_list = list(my_stack['instrument'])
    # print(my_stack_list)
    # print(my_stack)
    for x in stack['instrument'][:top_k]:
        # print(x)
        if num_buy >= max_buying:
            last_stock = my_stack_list[-1]
            num_last_stock = current_buying[last_stock][0]
            if num_last_stock == 1:
                del current_buying[last_stock]
                my_stack_list = my_stack_list[:-1]
            else:
                current_buying[last_stock][0] -= 1
            num_buy -= 1
            money += (current_buying[last_stock][2] - current_buying[last_stock][1]) / current_buying[last_stock][1] * money * 0.1
            # num_sell += 1
        if x not in current_buying:
            cur_stock = x
            cur_stock_inform = past_data[past_data['instrument'] == cur_stock]
            today_stock_price = cur_stock_inform.loc[str(idx).split(' ')[0]]['$open'].round(6)
            current_buying[x] = [1, today_stock_price, today_stock_price] #num, start_price, current_price
            num_buy +=1
        else:
            current_buying[x][0] += 1
            num_buy +=1
        # print(num_buy, current_buying)
        # input()
    
    today_profit = 1
    for cur in current_buying.values():
        num, start, end = cur
        profit = (end/start)**num
        today_profit *= profit
    today_profit = today_profit**(1/num_buy)
    
    if not np.isnan(today_profit):
        total_profit *= today_profit
    # total_profit = total_profit**(1/cnt)
    print("---------------------------------")
    print("day ", cnt, idx)
    print(num_buy, current_buying)
    print("today_profit: ", today_profit)
    print("total_profit: ", total_profit)
    print("current money: ", money)
    # print("------------------")
    # print(current_buying)
    # input()
    # clear_output()

---------------------------------
day  1 2015-01-02 00:00:00
5 {'GAS': [1, nan, nan], 'POM': [1, nan, nan], 'FCX': [1, 3.352878, 3.352878], 'APA': [1, 4.313486, 4.313486], 'FSLR': [1, 1.807599, 1.807599]}
today_profit:  nan
total_profit:  1.0
---------------------------------
day  2 2015-01-05 00:00:00
10 {'GAS': [2, nan, 1.846966], 'POM': [2, nan, 0.183334], 'FCX': [1, 3.352878, 3.312672], 'APA': [1, 4.313486, 4.312794], 'FSLR': [1, 1.807599, 1.773646], 'BKR': [1, 3.131421, 3.131421], 'BK': [1, 1.31369, 1.31369], 'GNW': [1, 0.455342, 0.455342]}
today_profit:  nan
total_profit:  1.0
---------------------------------
day  3 2015-01-06 00:00:00
10 {'GAS': [2, nan, 1.76781], 'FCX': [1, 3.352878, 3.187747], 'APA': [1, 4.313486, 4.144221], 'BK': [2, 1.31369, 1.28854], 'IBM': [1, 1.810677, 1.810677], 'GME': [1, 3.868295, 3.868295], 'RF': [1, 0.805594, 0.805594], 'MAT': [1, 3.721822, 3.721822]}
today_profit:  nan
total_profit:  1.0
---------------------------------
day  4 2015-01-07 00:00:00


KeyboardInterrupt: 